<a href="https://colab.research.google.com/github/SubhajoyMukherjee/AI-Tool-To-Manage-Customer-Orders-Data-Using-GEMINI-API/blob/main/AI_Tool_to_Extract_data_from_customer_orders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mail data Extraction
Steps of data extractio from GMAIL are :
1. Make sure IMAP is enabled in Gmail Setting
  (Log in in the Gmail Account and go to the Settings, See all settings, and select 'Forwarding and POP/IMAP' tab. In the "IMAP access" section, select Enable IMAP.)

2. If the two factor Authentification is enabled, it is required to create an application specific password that the code need to use.
  (Go to Gmail Account Setting and click on 'Security'. Scroll down to 2-step Verification. Select mail under Select App, and Other under Select Device. Give a name and the system gives you a password that you need to use to authenticate from python IMAP)

In [ ]:
#Importing libraries for Gamil data extraction
import imaplib as imap
import email
import yaml     #To load saved login credential from a yaml file



# The Below Code cell is to import deatils from .yaml file if the code is run in VSCODE/Local Device
###  Before running the Code Please remove the muti-line comments sign the two blocks and Follow the Commented instruction

In [ ]:
'''
#Save the MailId and Password to a .yml file to access the details through python code
#Give a Name of the .yml File. i.e. 'Gmail_Credential.yml'
#Inside the .yml File wite the MailId and Password in the below format(KEY-VALUE Pair Format):
      #user: "xyz@gmail.com"
      #password: "23415178"
#Then Access the details of the .yml File by the code and the path of the FILE
with open("C:/Users/subha/Downloads/Gmail_Credential.yml") as f:
  content=f.read()

'''

In [ ]:
'''
#From "Gmail_Credential.yml" file import username and password
my_credentials=yaml.load(content,Loader=yaml.FullLoader)

#Load the Username and pasword from yaml file
Username,Password=my_credentials['user'],my_credentials['password']   '''

#The Below Code cell is to import Gmail Credentials from Google Colab Secrets if the code is run in GOOGLE COLAB


In [ ]:
# Used to securely store your API key
from google.colab import userdata
Username=userdata.get('Username1')
Password=userdata.get('password1')


# Log in to Gmail using the credentials through Python and IMAP

In [ ]:
#URL for IMAP connection to GMAIL
imap_url='imap.gmail.com'

#Connection with gmail using SSL
my_mail=imap.IMAP4_SSL(imap_url)

#Log in to GMAIL using Credentials
my_mail.login(Username,Password)


#Fetch Mail Details/Mails from GMAIL Inbox

In [ ]:
#Select the INBOX to fetch messages
my_mail.select('Inbox')   #It will return the status of the process and also how many mails are there in the INBOX

#print(type(my_mail))  : <class 'imaplib.IMAP4_SSL'>


('OK', [b'653'])

In [ ]:
#Define the key value for email search

key='Subject'
value='order'

#Search for emails with specific key value
result,data=my_mail.search(None, key, value) #Here Data is the List of Mail Ids matches the Search Criteria
print(type(data))
print(data)

<class 'list'>
[b'626 627 629 630 631 632 633']


In [ ]:
#In the variable 'data' all the Mail Ids are stored as a Single string
#So to access each of the Mail Id from 'data' we need to split it
#IDs of all emails that we want to fetch
mail_id_list=data[0].split()

print(mail_id_list)
print(type(mail_id_list))

[b'626', b'627', b'629', b'630', b'631', b'632', b'633']
<class 'list'>


In [ ]:
msgs=[]   #Empty list to capture all message from Each Mail Ids from 'mail_id_list'

#Iterate through messages and Extract data into the 'msgs' list
for i in mail_id_list:
  typ,data=my_mail.fetch(i,'(RFC822)')    #RFC822 returns whole message
  msgs.append(data)

print(msgs)
print(type(msgs[0]))
print(msgs[0])

## Now We have all meassages but with a lot of details
So, we have to Extract the Main part fron the Email

In [ ]:
#In a multipart e-mail, email.message.Message.get_payload() returns a
# list with one item for each part. The easiest way is to walk the message
# and get the payload on each part:

#The Message object consists of headers and payloads where payload is the body of the emails

def data_extract_GMAIL():
  a=''
  for i in msgs[::-1]:
    for j in i:
      if(type(j)) is tuple:
        my_msg=email.message_from_bytes((j[1]))
        #print("_________________________________")
        a+='Subj: '
        a+=my_msg['subject']
        a+='\tfrom: '
        a+=my_msg['from']
        a+='\tDate: '
        a+=my_msg['date']
        a+='\tbody: '
        for part in my_msg.walk():
        #print(part.get_content_type())
          if part.get_content_type()=='text/plain':
            a+=part.get_payload()
        a+='\n'
  return a
#data=data_extract_GMAIL()
#print(data)
#print(data_extract_GMAIL())

# Extract data from PDF Files



In [ ]:
!pip install PyPDF2
!pip install fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=91e09b190eee335651f4d1f8cd5aa8aae04b7b63b90e47a3b26b8228c1469d74
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import PyPDF2 as pdf
from fpdf import FPDF

In [ ]:
def data_extract_PDF():
  a=input('Enter the PDF file name/path : ')
  reader=pdf.PdfReader(a)
  n=len(reader.pages)
  for i in range(n):
    a+=reader.pages[i].extract_text()
  return a

  #print(reader.pages[0].extract_text())
  #print(type(reader.pages[0].extract_text()))

#print(data)
print(type(data))


# Extract data from .TXT

In [ ]:
data=[]
def data_extract_TXT():
  a=input('Enter the TEXT file name/path : ')
  data=[]
  file=open(a,'r')
  content=file.readlines()
  content='\n'.join(content)
  #print(content)
  print(type(content))
  return content


# This is the setup for using of GEMINI API

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# List all gemini models which are there in google-generative-ai library
### 1. gemini-1.5-pro :
Inputs : Audio, images, videos, and text
Outputs : Text
Optimized for : Complex reasoning tasks such as code and text generation, text editing, problem solving, data extraction and generation
### gemini-1.5-flash :
Inputs : Audio, images, videos, and text
Outputs : Text
Optimied for : Fast and versatile performance across a diverse variety of tasks
### gemini-1.0-pro :
Inputs : Text
Outputs : Text
Optimized for : Natural language tasks, multi-turn text and code chat, and code generation
### gemini-pro-vision :
Inputs : Images, videos, and text
Outputs : Text
Optimized for : Visual-related tasks, like generating image descriptions or identifying objects in images
### text-embedding-004 :
Inputs : Text
Outputs : Text embeddings
Optimized for : Measuring the relatedness of text strings


In [ ]:
for n in genai.list_models():
  if 'generateContent' in n.supported_generation_methods:
    print(n.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## So, for NLP we use :
### gemini-1.0-pro-latest

In [ ]:
model=genai.GenerativeModel('gemini-1.0-pro-001')

# This is to be run before running the below GEMINI function if data is taken from .txt and .pdf files

In [ ]:
def DataExtractFromPDForTXTorGMAIL(z):
  if z==0:
    return data_extract_PDF()
  elif z==1:
    return data_extract_TXT()
  elif z==2:
    return data_extract_GMAIL()
  else:
    print("Please Enter Valid Number !!!!!!!")

z=int(input('''Press 0 to extract data from PDF File :
Press 1 to extract data from TEXT File :
Press 2 to extract data from GMAIL data : '''))

data=DataExtractFromPDForTXTorGMAIL(z)

Press 0 to extract data from PDF File :
Press 1 to extract data from TEXT File :
Press 2 to extract data from GMAIL data : 2


## This is the query which is to be passed to GEMINI with the extracted data to get the desired result

In [ ]:
Data_of_orders=data
Query='''In the above text Extract the food/vegetables names which are ordered, quatity of each order,date,email id , phone no. and also the payment method form the text.
 Make extract data in Comma Separated Values format and return it as in the comma separeted values format there will be six columns. And at first the columns name should be there
 The column names are strictly in the below format :
 Phone_no,Email_Id,Food_Name,Quantity,Payment_Method,Date. If any of the data is unavailable for each customer in the text then simply write it as 'NULL' in the CSV'''
print(type(data))

<class 'str'>


In [ ]:
response = model.generate_content(Data_of_orders + "     " + Query)
print(response.text)
print(type(response.text))
a=response.text

Phone_no,Email_Id,Food_Name,Quantity,Payment_Method,Date
NULL,subhajoymukherjee2003@gmail.com,potatoes,2kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,onions,1kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,cauliflower,1kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,chips,5 packets,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,cookies,3 packets,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,juice,2L bottle,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,eggs,2 dozen,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,milk,1L carton,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,apples,3kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,oranges,2kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,bananas,1kg,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,basmati rice,1kg,Netbanking,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,urad dal,500g,NULL,10 Jun 2024
NULL,subhajoymukherjee2003@gmail.com,green moong da

In [ ]:
a=response.text

# Save the Gemini Output to a CSV File

In [ ]:
import csv
from csv import writer
import pandas as pd

In [ ]:
def edit_csv(a):
  if v==0:
    with open('Example.csv','w')as out:
      out.write(a)
      out.close()
  else:
    df1=pd.DataFrame()
    df1=pd.read_csv('Example.csv')
    with open('Example.csv','w')as out:
      out.write(a)
      out.close()
    df2=pd.read_csv('Example.csv')
    df3=pd.concat([df1,df2],ignore_index=True)
    df3.to_csv('Example.csv')
v=int(input('''Press 0 to make a fresh CSV file with the data :
Press 1 to add the data in the existing CSV file :'''))
edit_csv(a)







Press 0 to make a fresh CSV file with the data :
Press 1 to add the data in the existing CSV file :1
